In [1]:
#建立連結
from py2neo import  Graph, Node, Relationship, NodeMatcher, RelationshipMatcher
graph = Graph(scheme="bolt", host="localhost", port=7687,secure=True, auth=('shao', 'shao')) #auth改成自己的帳密

In [2]:
#建立Course Node
with open("./Course_noIntro.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        ClassName,ClassID,ClassData,ClassPre,ClassStop,ClassTeacher,ClassSchool=c.split(",")
        ClassName=ClassName.strip()
        node=Node("Courses",name=ClassName,ID=ClassID,data=ClassData,pre=ClassPre,stop=ClassStop,teacher=ClassTeacher,school=ClassSchool.replace('\n', ''))
        graph.create(node)

In [3]:
#建立Professor Node
with open("./Professor.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        professorID, professorName, professorDepartment,professorSchool = c.split(",")
        node=Node("Professors",name=professorName,ID=professorID,Department=professorDepartment,School=professorSchool.replace('\n',''))
        graph.create(node)

In [4]:
#建立Student Node
with open("./OnlyStudent.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        StudentID, StudentName = c.split(",")
        StudentName=StudentName.rstrip()
        node=Node("Students",name=StudentName,ID=StudentID,password="000000")
        graph.create(node)

In [5]:
#建立Courses Relation
with open("./Course_noIntro.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        ClassName,ClassID,ClassData,ClassPre,ClassStop,ClassTeacher,ClassSchool=c.split(",")
        #先修
        PreClass=[]
        eachPreClass=ClassPre.split("、")
        for i in eachPreClass:
            if(len(i)<3):
                break
            PreClass.append(i)
        #擋修
        StopClass=[]
        eachClass=ClassStop.split("、")
        for i in eachClass:
            if(len(i)<3):
                break
            StopClass.append(i)
        
        matcher = NodeMatcher(graph)
        #處理先修
        for i in PreClass:     
            existing_u1 = matcher.match("Courses", name=ClassName).first()
            existing_u2 = matcher.match("Courses", name=i).first()
            existing_u1_pre_u2 = Relationship(existing_u1, '先修', existing_u2)
            graph.create(existing_u1_pre_u2)
        #處理擋修
        for i in StopClass:
            existing_u1 = matcher.match("Courses", name=ClassName).first()
            existing_u2 = matcher.match("Courses", name=i).first()
            existing_u1_stop_u2 = Relationship(existing_u1, '擋修', existing_u2)
            graph.create(existing_u1_stop_u2)
        #處理教授
        existing_u1 = matcher.match("Professors", name=ClassTeacher).first()
        existing_u2 = matcher.match("Courses", teacher=ClassTeacher)
        result = list(existing_u2) 
        for i in range(len(result)):
            existing_u1_teach_u2 = Relationship(existing_u1, '教', result[i])
            graph.create(existing_u1_teach_u2)

In [6]:
#建立Student Relation
with open("./Student.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        StudentID, StudentName, ClassName, ClassTime, StudentSchool = c.split(",")
        StudentName=StudentName.rstrip()
        ClassName=ClassName.rstrip()
        existing_u1 = matcher.match("Students", name = StudentName).first()
        existing_u2 = matcher.match("Courses", name = ClassName).first()  
        existing_u1_teach_u2 = Relationship(existing_u1, '修', existing_u2,date=ClassTime)
        graph.create(existing_u1_teach_u2)

In [ ]:


while(1):
    getNumber=input("請輸入對應號碼(0為基本查詢,1為進階查詢,2為使用者相關,其他則跳出系統):")
    if(getNumber=="0"):
        getInner=input("請輸入查詢類型(0為課程名稱查詢,1為教授查詢,2為學生查詢,其他則回到上一層)：")
        if(getInner=="0"):
            getName=input("請輸入查詢課程名稱:")
            getName=getName.strip()
            existing_u1 = matcher.match("Courses", name = getName).first()
            print('課程代碼:' + existing_u1['ID'])
            print('課程名稱:' + existing_u1['name'])
            print('課程網頁:' + existing_u1['data'])
            print('先修課程:' + existing_u1['pre'])
            print('擋修課程:' + existing_u1['stop'])
            print('上課教授:' + existing_u1['teacher'])
            print('授課學校:' + existing_u1['school'])
        elif(getInner=="1"):
            getName=input("請輸入查詢教授姓名:")
            getName=getName.strip()  
            existing_u1 = matcher.match("Professors", name = getName).first()
            tempList=""
            for rel in (graph.match((existing_u1, None), "教")):
                if(tempList==""):
                    tempList=tempList+rel.end_node['name']
                else:
                    tempList=tempList+','+rel.end_node['name']                    
            print("教授代碼:" + existing_u1['ID'])  
            print("教授姓名:" + existing_u1['name']) 
            print("教授學校:" + existing_u1['School'])
            print("學校部門:" + existing_u1['Department']) 
            print("教授課程:" + str(tempList))
        elif(getInner=="2"):
            getName=input("請輸入查詢學生姓名:")
            getName=getName.strip()  
            existing_u1 = matcher.match("Students", name = getName).first()
            tempList=""
            for rel in (graph.match((existing_u1, None), "修")):
                if(tempList==""):
                    tempList=tempList+rel.end_node['name']
                else:
                    tempList=tempList+','+rel.end_node['name']
            print("學生學號:" + existing_u1['ID'])
            print("學生姓名:" + existing_u1['name'])
            print("已修課程:" + str(tempList))
    elif(getNumber=="1"):
        getInner=input("請輸入查詢類型(0為查詢系列課程先修、擋修,1為查詢該堂修課學生,2為學生查詢,其他則回到上一層)：")
        print('進階查詢')
    elif(getNumber=="2"):
        print('使用者相關')
    else:
        break

請輸入對應號碼(0為基本查詢,1為進階查詢,2為使用者相關,其他則跳出系統):0
請輸入查詢類型(0為課程名稱查詢,1為教授查詢,2為學生查詢,其他則回到上一層)：2
請輸入查詢學生姓名:楚留香
學生學號:10000064
學生姓名:楚留香
已修課程:物件導向程式設計,物件導向設計,計算機程式設計,計算機程式設計實習,計算機概論,離散數學
